### Understanding InMemory in AutoGen Agents / Teams

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core import CancellationToken
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
import os

from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

if not all([azure_openai_api_key, azure_openai_endpoint, azure_openai_deployment_name, azure_openai_api_version]):
    raise ValueError("""
        Please set all required environment variables: 
            AZURE_OPENAI_API_KEY, 
            AZURE_OPENAI_ENDPOINT, 
            AZURE_OPENAI_DEPLOYMENT_NAME, 
            AZURE_OPENAI_API_VERSION
        """)


In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [ ]:
chroma_user_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="preferences",
        persistent_path="./chroma_db",
        k=5,
        score_threshold=0.1,
    )
)

await chroma_user_memory.clear()

In [ ]:
await chroma_user_memory.add(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content="The weather should be in metric units, such as Celsius for temperature and kilometers per hour for wind speed.",
    )
)

await chroma_user_memory.add(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content="Meal Recipe must be vegan, and the ingredients should be listed in grams.",
    )
)

await chroma_user_memory.add(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content="I live in London, UK, and I prefer to receive information in British English.",
    )
)

In [ ]:
preferences = await chroma_user_memory.query(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content="What are my preferences?",
    )
)

print(preferences)

In [ ]:
chroma_user_memory.dump_component().model_dump_json()

In [ ]:
async def get_weather_info(location: str, units: str = "imperial") -> str:
    if units == "metric":
        return f"The weather in {location} is 20°C with a wind speed of 15 km/h."
    elif units == "imperial":
        return f"The weather in {location} is 68°F with a wind speed of 9 mph."
    else:
        return f"Sorry, I don't have weather information for {location} in {units} units."

assistant_agent = AssistantAgent(
    name="WeatherAndMealRecipeAssistant",
    model_client=model_client,
    memory=[chroma_user_memory],
    tools=[get_weather_info],
)

In [ ]:
stream = assistant_agent.run_stream(
    task = "What is weather in my location London?"
)

await Console(stream)

In [ ]:
stream = assistant_agent.run_stream(
    task = "What is weather in New York?"
)

await Console(stream)

In [ ]:
stream = assistant_agent.run_stream(
    task = "Can you suggest me a recipe with Broth?"
)

await Console(stream)